<a href="https://colab.research.google.com/github/Nganle21/Big-Data-Dreamteam/blob/main/Ngan/PF_songfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import re

cid = '6c8892a3331d4602830359f009d42765'
secret = '2b147051b73d415c891fccec0d139b15'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [51]:
#Artist details
artist_name= 'Tyga'
artistID = (sp.search(q='artist:' + artist_name, type='artist'))['artists']['items'][0]['id']
artistName = (sp.search(q='artist:' + artist_name, type='artist'))['artists']['items'][0]['name']
artistGenre  = (sp.search(q='artist:' + artist_name, type='artist'))['artists']['items'][0]['genres']

#search for album
albumResults = sp.artist_albums(artistID, limit=50)
albumResults = albumResults['items']
z=0
tracks_info = []

#Create DF to store data
df_artists = pd.DataFrame()
df_albums = pd.DataFrame()

# Create a empty set 
artists_searched = set()
artists_searched.add(artist_name)
        
## Extract artist data and fill database
df_artists = df_artists.append({'artist_id': artistID,
                                    'artist_name':artistName,
                                    'genres':artistGenre},
                                    ignore_index=True
                                       )

# Loop over album
for album in albumResults:
            
    ## Extract album data and fill database
    albumID = album['id']
    album_name =  album['name']
    df_albums = df_albums.append({'artist_name':album['artists'][0]['name'],
                                          'artist_id':album['artists'][0]['id'],
                                          'album_id': album['id'],
                                          'album_name':album['name'],
                                          'release_date':album['release_date'],
                                          'total_tracks':album['total_tracks'],
                                          'type':album['album_group']},
                                          ignore_index=True
                                         )     
df_albums= df_albums.astype({'total_tracks':int}) 
df_albums= df_albums.drop_duplicates(subset=['album_name'])

#Deleting duplicated albums
k=0
Not_good_words=['Remix','Edition','Accoustic','Version','Deluxe','Acoustic']
TODEL=[]
for alb in df_albums.album_name.drop_duplicates().tolist():
    alb_sp=re.split('(\W+)',alb)
    if len(alb_sp)!=0:
        for s in alb_sp:
            if s in Not_good_words:
                TODEL.append(k)
    k=k+1
alb=np.array(df_albums.album_name.drop_duplicates().tolist())
#Taking the most significative ones

ALB=[]
for a in range(len(alb)):
    if a not in TODEL:
        ALB.append(alb[a])
LIST_ALB_TITLE=[]
for a in ALB:
    alb_sp=a.split()
    LIST_ALB_TITLE.append(sp)

df_albums=df_albums[(df_albums.album_name.isin(ALB))]
    


In [52]:
df_artists.head()

,artist_id,artist_name,genres
0,5LHRHt1k9lMyONurDHEdrp,Tyga,"[dance pop, hip hop, pop, pop rap, rap, southe..."


In [53]:
df_albums.head()

,artist_name,artist_id,album_id,album_name,release_date,total_tracks,type
1,Tyga,5LHRHt1k9lMyONurDHEdrp,6hHIX3lfGKnZ2ji41YZMVV,Legendary,2019-06-07,14,album
3,Tyga,5LHRHt1k9lMyONurDHEdrp,2AoE3mTcrazotgJ8PTwxOM,Kyoto,2018-02-16,14,album
4,Tyga,5LHRHt1k9lMyONurDHEdrp,6lWQVd4FpHVh2MCGHxwIsd,Bugatti Raww,2017-10-23,10,album
5,Tyga,5LHRHt1k9lMyONurDHEdrp,4BqsBWrnGJQHSgmA9xk5rh,Bitch I'm the Shit 2,2017-07-21,15,album
6,Tyga,5LHRHt1k9lMyONurDHEdrp,6MDoqpC29hTr48aHe3Pw7n,B*tch I'm the Sh*t 2,2017-07-21,15,album


In [54]:
for id in df_albums['album_id']:
    # TRACK DETAILS
    trackResults = sp.album_tracks(id)
    trackResults = trackResults['items']
        
    ## Loop over tracks
    for track in trackResults:
                
        artists_names =[]
        artists_ids = []
        artists_genres = []
        
        for artist in track['artists']:
            artists_names.append(artist['name'])
            artists_ids.append(artist['id'])
            artists_genres.append(artistGenre)
                
            ## Extract track data and fill database
        if (artistID in artists_ids or artist_name in artists_names):
            z+=1
            track_info = [id,artists_ids[0],  
                            artists_names[0], 
                            track['uri'],
                            track['name'],
                            artists_genres[0]]
            tracks_info.append(track_info)
#create df_tracks df and drop duplicates
df_tracks = pd.DataFrame(data = tracks_info,columns=['album_id','artist_id','artist_name','track_uri','track_name','genres'])
df_tracks=df_tracks.drop_duplicates(subset=['track_name','track_uri'])
#join with df_album for release dates
df_tracks =(pd.merge(df_tracks,df_albums,how ='left',left_on = 'album_id',right_on='album_id',suffixes=('', '_y')))
df_tracks = df_tracks[df_tracks.columns.drop(list(df_tracks.filter(regex='_y')))]



In [55]:
df_tracks.head()

,album_id,artist_id,artist_name,track_uri,track_name,genres,album_name,release_date,total_tracks,type
0,6hHIX3lfGKnZ2ji41YZMVV,5LHRHt1k9lMyONurDHEdrp,Tyga,spotify:track:0CR60XwKbzgohE3I6NMjlR,Too Many,"[dance pop, hip hop, pop, pop rap, rap, southe...",Legendary,2019-06-07,14,album
1,6hHIX3lfGKnZ2ji41YZMVV,5LHRHt1k9lMyONurDHEdrp,Tyga,spotify:track:1zZEly0sLmyDm75c1pAqYf,Lightskin Lil Wayne,"[dance pop, hip hop, pop, pop rap, rap, southe...",Legendary,2019-06-07,14,album
2,6hHIX3lfGKnZ2ji41YZMVV,5LHRHt1k9lMyONurDHEdrp,Tyga,spotify:track:1msycdMqTjoKvrP3WiHMCQ,On Me (feat. Lil Wayne),"[dance pop, hip hop, pop, pop rap, rap, southe...",Legendary,2019-06-07,14,album
3,6hHIX3lfGKnZ2ji41YZMVV,5LHRHt1k9lMyONurDHEdrp,Tyga,spotify:track:1bFlyIMhjyZavdg5PXrkLs,Stash (feat. Blueface),"[dance pop, hip hop, pop, pop rap, rap, southe...",Legendary,2019-06-07,14,album
4,6hHIX3lfGKnZ2ji41YZMVV,5LHRHt1k9lMyONurDHEdrp,Tyga,spotify:track:1jlnBQGP2DtSEhMgQaz7a7,Haute (feat. J Balvin & Chris Brown),"[dance pop, hip hop, pop, pop rap, rap, southe...",Legendary,2019-06-07,14,album


In [56]:
audio_features = []
for uri in df_tracks['track_uri'].values:
    audio_features.append(sp.audio_features(uri)[0])
audio_features = pd.DataFrame(audio_features)[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                             'acousticness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
audio_features['duration'] = audio_features['duration_ms']/60000
audio_features = pd.concat([df_tracks['track_name'],df_tracks['track_uri'],df_tracks['artist_name'],df_tracks['release_date'],df_tracks['genres'],audio_features],axis=1)
audio_features.head()

,track_name,track_uri,artist_name,release_date,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature,duration
0,Too Many,spotify:track:0CR60XwKbzgohE3I6NMjlR,Tyga,2019-06-07,"[dance pop, hip hop, pop, pop rap, rap, southe...",0.794,0.661,2,-4.188,1,0.263,0.0488,0.2140,0.359,99.977,145293,4,2.421550
1,Lightskin Lil Wayne,spotify:track:1zZEly0sLmyDm75c1pAqYf,Tyga,2019-06-07,"[dance pop, hip hop, pop, pop rap, rap, southe...",0.916,0.518,8,-7.082,1,0.475,0.3110,0.1380,0.464,99.983,153575,4,2.559583
2,On Me (feat. Lil Wayne),spotify:track:1msycdMqTjoKvrP3WiHMCQ,Tyga,2019-06-07,"[dance pop, hip hop, pop, pop rap, rap, southe...",0.669,0.753,1,-4.054,1,0.341,0.2900,0.6280,0.398,95.953,162447,4,2.707450
3,Stash (feat. Blueface),spotify:track:1bFlyIMhjyZavdg5PXrkLs,Tyga,2019-06-07,"[dance pop, hip hop, pop, pop rap, rap, southe...",0.693,0.607,4,-4.724,0,0.352,0.3190,0.1100,0.607,98.969,174536,4,2.908933
4,Haute (feat. J Balvin & Chris Brown),spotify:track:1jlnBQGP2DtSEhMgQaz7a7,Tyga,2019-06-07,"[dance pop, hip hop, pop, pop rap, rap, southe...",0.946,0.723,9,-3.692,1,0.114,0.0778,0.0608,0.349,102.956,160877,4,2.681283
